In [ ]:
# Uncomment line below to install exlib
# !pip install exlib

In [1]:
import torch
import yaml
import argparse
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import exlib
import math
import torch.nn.functional as F

from datasets import load_dataset
from collections import namedtuple
from exlib.datasets.supernova import SupernovaDataset, SupernovaClsModel, SupernovaFIXScores, get_supernova_scores
from exlib.features.time_series.chunk import BaselineGroups
from exlib.utils.supernova_helper import *
from tqdm.auto import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

/opt/conda/lib/python3.10/site-packages/huggingface_hub-0.23.2-py3.8.egg/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.


### Overview
* The objective is to classify astronomical sources that vary with time into different classes

### Load datasets and pre-trained models

In [2]:
test_dataset = SupernovaDataset(data_dir = "BrachioLab/supernova-timeseries", split="test")
model = SupernovaClsModel(model_path = "BrachioLab/supernova-classification")

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


num labels: 14
Using Fourier PE
classifier dropout: 0.2


### Model prediction

In [3]:
model = model.to(device)
test_dataloader = create_test_dataloader(
    dataset=test_dataset,
    batch_size=5,
    compute_loss=True
)

original dataset size: 792
remove nans dataset size: 792


In [4]:
# model prediction
with torch.no_grad():
    y_true = []
    y_pred = []
    alignment_scores_all = []
    for bi, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items() if k != "objid"}
        outputs = model(**batch)
        y_true.extend(batch['labels'].cpu().numpy())
        y_pred.extend(torch.argmax(outputs.logits, dim=2).squeeze().cpu().numpy())
# model prediction
print(f"accuracy: {sum([1 for i, j in zip(y_true, y_pred) if i == j]) / len(y_true)}")

  0%|          | 0/159 [00:00<?, ?it/s]

accuracy: 0.7967171717171717


### Feature alignment

In [5]:
test_dataloader = create_test_dataloader_raw(
    dataset=test_dataset,
    batch_size=5,
    compute_loss=True
)

original dataset size: 792
remove nans dataset size: 792


In [6]:
fix_score_all = 0
elements_all = 0
with torch.no_grad():
    for bi, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        
        BaselineGroup = BaselineGroups(ngroups=10, window_size=100)
        pred_groups = BaselineGroup(**batch)

        SupernovaFIXScore = SupernovaFIXScores(sigma=1, nchunk=7, groups = pred_groups)
        fix_score = SupernovaFIXScore(**batch)
        fix_score_all += fix_score.sum().item()
        elements_all += fix_score.numel()
fix_score_mean = fix_score_all / elements_all
print(f"fix score: {fix_score_mean}")

  0%|          | 0/159 [00:00<?, ?it/s]

fix score: 0.055455425500192425


### Baselines
- 5 chunks
- 10 chunks
- 15 chunks

In [8]:
scores = get_supernova_scores(dataset=test_dataset, batch_size = 5)

original dataset size: 792
remove nans dataset size: 792


  0%|          | 0/159 [00:00<?, ?it/s]

Avg alignment of 5 features: 0.0337
Avg alignment of 10 features: 0.0555
Avg alignment of 15 features: 0.0554
